In [3]:
import pandas as pd

In [7]:
assessment_df = pd.read_csv('Cleaned_Assessment.csv')
diagnosis_df = pd.read_csv('Diagnosis.csv')
assessment_df.head()

,fFolder,PatientNum,Description,Question_Name,Answer_Text
0,A,640001,מגורים ותמיכה חברתית,מצב מגורים,מוסד
1,A,640001,מגורים ותמיכה חברתית,תמיכה,מתגורר בבית אבות
2,A,640001,אוכלוסיה בסיכון,מצב הכרה,בהכרה
3,A,640001,אוכלוסיה בסיכון,מצב קוגניטיבי,"מתמצא (בזמן, במקום, באנשים)"
4,A,640001,אוכלוסיה בסיכון,מצב ריגשי,תקין


In [8]:
diagnosis_df.columns

Index(['fFolder', 'PatientNum', 'AdmissionNum', 'TransferNum', 'Description',
       'Entry_Date', 'MP', 'SP', 'Recurrent', 'Side', 'Name', 'ICD9'],
      dtype='object')

In [ ]:
# --- Diagnosis Data Exploration ---
# This cell explores the Diagnosis.csv file to identify the most common
# chronic conditions, which can be used to create more sophisticated facts.

print("--- Diagnosis Data Exploration ---")
try:
    diagnosis_df = pd.read_csv('Diagnosis.csv')
    
    # Filter for background diagnoses (fFolder = 'X') as these represent chronic conditions
    background_diagnoses_df = diagnosis_df[diagnosis_df['fFolder'] == 'X']
    
    if not background_diagnoses_df.empty:
        # Count the occurrences of each diagnosis name
        common_diagnoses = background_diagnoses_df['Name'].value_counts()
        
        print("\n--- Top 20 Most Common Background Diagnoses ---")
        print(common_diagnoses.head(20))
        
    else:
        print("No background diagnoses (fFolder = 'X') found in Diagnosis.csv.")

except FileNotFoundError:
    print("Error: Diagnosis.csv not found.")
except Exception as e:
    print(f"An error occurred: {e}")